# Ultralytics YOLO format

## 1. Directory

```powershell
<Custom Dataset Directory>
****├── 0.jpg
├── 0.txt
└── ...
```

### Split Dataset Directory

```powershell
rico_yolo
├── test
│   ├── 0.jpg
│   ├── 0.txt
│   └── ...
├── train
│   ├── 2.jpg
│   ├── 2.txt
│   └── ...
└── val
    ├── 36.jpg
    ├── 36.txt
    └── ...
```

## 2. Annotation

```
<object-class> <x> <y> <width> <height>
```

### 예시) 2.txt

13 0.5 0.4833984375 0.7944444444444444 0.626171875
3 0.5 0.758203125 0.7944444444444444 0.0765625
21 0.7875 0.758203125 0.16111111111111112 0.065625
6 0.5 0.4451171875 0.7944444444444444 0.549609375
21 0.5 0.198046875 0.6777777777777778 0.05546875
21 0.35381944444444446 0.2568359375 0.3854166666666667 0.062109375
8 0.2013888888888889 0.358984375 0.11666666666666667 0.065625
8 0.7986111111111112 0.358984375 0.11666666666666667 0.065625

## Dataset File Format

In [ ]:
yaml_text = """
train: /tf/notebook/arrow/train
val: /tf/notebook/arrow/val
test: /tf/notebook/arrow/test

nc: 20
names:
    ['',
    'typo_price_num',
    'typo_price_dollar',
    'typo_price_W',
    'typo_price_won_en',
    'typo_price_won_kr',
    'image_photo',
    'image_colorBG',
    'image_removeBG',
    'icon_arrow_left',
    'icon_arrow_top',
    'icon_arrow_bottom',
    'icon_arrow_right',
    'icon_video_play',
    'icon_SNS_insta',
    'icon_SNS_youtube',
    'btn_radius',
    'btn_ellipse',
    'btn_square',
    'typo_text']"""

with open('/tf/notebook/arrow/data.yaml', 'w') as file:
    file.write(yaml_text)

In [ ]:
### 작업 디렉토리

'''powershell
weven
├── datasets
│   └── Obejectblock
└── notebooks
    └── read_rico_dataset.ipynb

'''
# Import Libraries

In [ ]:
# Step 3 Training - Preparing  Dataset for YOLO




from pathlib import Path
import json
import functools
import itertools
import shutil
import numpy as np
import pandas as pd
from PIL import Image




## For parallel computing


In [ ]:
import concurrent.futures
num_workers = 32  # The number of threads or processes

# mt_executor = concurrent.futures.ThreadPoolExecutor(num_workers)  
# Use ThreadPoolExecutor for CPU-bound tasks.
# mp_executor = concurrent.futures.ProcessPoolExecutor(num_workers)  
# Use ProcessPoolExecutor for I/O-bound tasks.

#  dataset path settings

In [ ]:
screenshots_path = Path('../datasets/Object/~')  # 66261 '.jpg' and '.json'
annotations_path = Path('../datasets/Object/~/')  # 66261 '.png' and 'json'

image_ids = pd.Series([int(p.stem) for p in screenshots_path.iterdir() if p.suffix == '.json'], name='image_id')
image_ids = image_ids.sort_values().reset_index(drop='index')

screenshot_paths = [screenshots_path / f'{image_id}.jpg' for image_id in image_ids]
annotation_paths = [annotations_path / f'{image_id}.png' for image_id in image_ids]
view_hierarchy_json_paths = [screenshots_path / f'{image_id}.json' for image_id in image_ids]
annotation_hierarchy_json_paths = [annotations_path / f'{image_id}.json' for image_id in image_ids]





## Read hierarchy json files


In [ ]:
def get_hierarchy_from_json(json_path):
    with open(json_path) as fp:
        hierarchy = json.load(fp)
        return hierarchy

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    annotation_hierarchies = mp_executor.map(get_hierarchy_from_json, annotation_hierarchy_json_paths)
    annotation_hierarchies = list(annotation_hierarchies)

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    view_hierarchies = mp_executor.map(get_hierarchy_from_json, view_hierarchy_json_paths)
    view_hierarchies = list(view_hierarchies)

# Get Image sizes (가로, 세로)


In [ ]:
def get_imsize(path):
    im = Image.open(path)
    return im.size

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    screenshot_sizes = mp_executor.map(get_imsize, screenshot_paths)
    screenshot_sizes = list(screenshot_sizes)

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    annotation_sizes = mp_executor.map(get_imsize, annotation_paths)
    annotation_sizes = list(annotation_sizes)

screenshot_sizes_df = pd.DataFrame(screenshot_sizes, columns=['width', 'height'])
annotation_sizes_df = pd.DataFrame(annotation_sizes, columns=['width', 'height'])

screenshot_sizes_df.insert(0, 'image_id', image_ids)
annotation_sizes_df.insert(0, 'image_id', image_ids)
screenshot_sizes_df['ratio'] = screenshot_sizes_df['width'] / screenshot_sizes_df['height']



# Get useful information from hierarchies

## A. From view_hierarchies

### 1. root size


In [ ]:
def get_root_size(hierarchy):
    return hierarchy['activity']['root']['rel-bounds']

root_sizes = map(get_root_size, view_hierarchies)

root_sizes_df = pd.DataFrame(root_sizes, columns=['x1', 'y1', 'x2', 'y2'])
root_sizes_df.insert(0, 'image_id', image_ids)
root_sizes_df['ratio'] = root_sizes_df['x2'] / root_sizes_df['y2']






### 2. root classes


In [ ]:
def get_root_classes(hierarchy):
    root_class_name = hierarchy['activity']['root']['class']
    return root_class_name

root_class_names = list(map(get_root_classes, view_hierarchies))
pd.Series(root_class_names).value_counts()





## B. From annotaion_hierarchies



In [ ]:
def get_unique_attributes(hierarchy, attribute_name):
    uniques = set()

    recursive_function = functools.partial(get_unique_attributes, attribute_name=attribute_name)
    if type(hierarchy) is list:
        uniques_list = map(recursive_function, hierarchy)
        uniques |= functools.reduce(lambda x,y: x | y, uniques_list, set())
    
    if type(hierarchy) is dict:
        if attribute_name in hierarchy:
            uniques.add(hierarchy[attribute_name])
    
        if 'children' in hierarchy:
            uniques |= recursive_function(hierarchy['children'])
    
    return uniques



### 1. componentLabel, 2. iconClass, 3.textButtonClass

In [ ]:
all_component_labels = get_unique_attributes(annotation_hierarchies, 'componentLabel')
icon_classes = get_unique_attributes(annotation_hierarchies, 'iconClass')
text_button_classes = get_unique_attributes(annotation_hierarchies, 'textButtonClass')

all_component_labels = sorted(list(all_component_labels))
icon_classes = sorted(list(icon_classes))
text_button_classes = sorted(list(text_button_classes))


# Get bounding boxes


In [ ]:
def get_component_and_bounds(hierarchy):
    bboxes = []

    if 'componentLabel' in hierarchy:
        label = hierarchy['componentLabel']
        x1, y1, x2, y2 = hierarchy['bounds']
        bboxes.append(dict(label=label, x1=x1, y1=y1, x2=x2, y2=y2))

    if 'children' in hierarchy:
        bboxes += list(itertools.chain.from_iterable(map(get_component_and_bounds, hierarchy['children'])))
    
    return bboxes

all_bboxes = list(map(get_component_and_bounds, annotation_hierarchies))


# Preprocess dataset

### Drop duplicated boxes



In [ ]:

all_bbox_dfs = list(map(lambda x: pd.DataFrame(x).drop_duplicates().reset_index(drop=True), all_bboxes))



all_bbox_df = pd.concat(all_bbox_dfs, keys=image_ids, names=['image_id', 'bbox_id'])
all_bbox_df = all_bbox_df.convert_dtypes()

### **Find landscape view (wrong matches)**


In [ ]:
landscape_screenshot_df = screenshot_sizes_df[screenshot_sizes_df['ratio'] > 1]
landscaped = all_bbox_df.index.isin(landscape_screenshot_df['image_id'], level='image_id')



### **Find overflowed boxes**


In [ ]:

annotation_sizes = {'width': 1440, 'height': 2560}
overflowed = all_bbox_df['x1'] < 0
overflowed |= all_bbox_df['y1'] < 0
overflowed |= all_bbox_df['x2'] > annotation_sizes['width']
overflowed |= all_bbox_df['y2'] > annotation_sizes['height']

### **Find boxes with 0 or negative values**

In [ ]:

negated = all_bbox_df['x1'] >= all_bbox_df['x2']
negated |= all_bbox_df['y1'] >= all_bbox_df['y2']


### **Filter out all corrupted boxes**

In [ ]:
bbox_df = all_bbox_df[~(landscaped | overflowed | negated)]

# **Convert to YOLO format**

In [ ]:
class_index = pd.Series(list(all_component_labels))
class_index = pd.Series(class_index.index.values, index=class_index)



yolo_bbox_df = pd.DataFrame({
    'label': pd.Series(class_index[bbox_df['label']].values, index=bbox_df.index),
    'x': (bbox_df['x1'] + bbox_df['x2']) / 2 / annotation_sizes['width'],
    'y': (bbox_df['y1'] + bbox_df['y2']) / 2 / annotation_sizes['height'],
    'width': (bbox_df['x2'] - bbox_df['x1']) / annotation_sizes['width'],
    'height': (bbox_df['y2'] - bbox_df['y1']) / annotation_sizes['height'],
})


# Create YOLO dataset

In [ ]:
object_yolo_path = Path('tf/datasets/datasets')
object_yolo_path.mkdir(parents=True, exist_ok=True)

### Create txt file for each image


In [ ]:
for image_id, image_bboxes_df in yolo_bbox_df.groupby('image_id'):
    image_bboxes_df.to_csv(object_yolo_path / f'{image_id}.txt', sep=' ', header=False, index=False)

### Create dataset yaml file

In [ ]:
!

In [ ]:
import os
import shutil
import random

# 데이터셋 폴더 경로
dataset_path = '/tf/notebook/Object_Detect/datasets'
train_path = 'train'
test_path = 'test'
valid_path = 'valid'

# train, test, valid 폴더 생성
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)

# 데이터셋 파일 리스트 생성
file_list = os.listdir(dataset_path)

# 파일 리스트를 무작위로 섞음
random.shuffle(file_list)

# 데이터셋을 train:test:valid = 70:15:15로 나눔
train_size = int(0.7 * len(file_list))
test_size = int(0.15 * len(file_list))



In [4]:
import os
import shutil
import random

# 데이터셋 폴더 경로
dataset_path = '/tf/notebook/datasetstest/'
train_path = '/tf/notebook/datasetstest/train'
test_path = '/tf/notebook/datasetstest/test'
valid_path = '/tf/notebook/datasetstest/val'

# train, test, valid 폴더 생성
os.makedirs(train_path + '/images', exist_ok=True)
os.makedirs(train_path + '/labels', exist_ok=True)
os.makedirs(test_path + '/images', exist_ok=True)
os.makedirs(test_path + '/labels', exist_ok=True)
os.makedirs(valid_path + '/images', exist_ok=True)
os.makedirs(valid_path + '/labels', exist_ok=True)

# 데이터셋 파일 리스트 생성
file_list = os.listdir(dataset_path)
random.shuffle(file_list)

# 데이터셋을 train:test:valid = 70:15:15로 나눔
train_size = int(0.8 * len(file_list))
test_size = int(0.1 * len(file_list))
valid_size = int(0.1 * len(file_list))
# train 데이터 복사
for filename in file_list[:train_size]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(train_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(train_path + '/labels', filename.replace('.jpeg', '.txt')))

# test 데이터 복사
for filename in file_list[train_size:train_size+test_size]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(test_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(test_path + '/labels', filename.replace('.jpeg', '.txt')))

# validation 데이터 복사
for filename in file_list[train_size+test_size:]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(valid_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(valid_path + '/labels', filename.replace('.jpeg', '.txt')))

In [ ]:
splits = ['train', 'val', 'test']
with open('data.yaml', 'w') as f:
    for split in splits:
        split_path = (object_yolo_path.parent / split).resolve()
        f.write(f'{split}: {split_path}\n')
    
    num_labels = len(all_component_labels)

    f.write('names:\n')
    for index, label in enumerate(all_component_labels):
        f.write(f'  {index}: {label}\n')

# Copy screenshot files to object_yolopath

In [ ]:
image_ids_to_copy = yolo_bbox_df.index.get_level_values('image_id').unique()

def copy_image(image_id):
    shutil.copy(screenshots_path / f'{image_id}.jpg', rico_yolo_path / f'{image_id}.jpg')
    return True

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    results = list(mp_executor.map(copy_image, image_ids_to_copy))

# Data Split yolo

### 데이터셋 path 설정

In [ ]:
object_yolo_path = Path('../datasets/object_yolo/all')

# Image Id get

In [ ]:
all_image_ids = map(lambda p :int(p.stem), object_yolo_path.iterdir())
all_image_ids = pd.Series(all_image_ids).unique()

# Random array create

In [ ]:
num_samples = len(all_image_ids)
seed_id = 0
np.ramdom.sedd(seed_id)
random_indices = np.random.choice(num_samples, num_samples, replace=False)

# Data Split

In [ ]:
splits = ['train', 'val', 'test']
split_ratio = [0.8, 0.1, 0.1]

## 랜덤 배열을 비율에 맞추어 나누기

In [ ]:
ind = np.array(split_ratio).cumsum() * num_samples
ind = np.floor(ind).astype(int) - 1

## split별 id값 얻어내기

In [ ]:
image_ids = {
    'train': all_image_ids[:ind[0]],
    'val': all_image_ids[ind[0]:ind[1]],
    'test': all_image_ids[ind[1]:],
}

## 각 split 폴더로 복사하기

In [ ]:
for split in splits:
    object_yolo_split_path = object_yolo_path.parent / split
    object_yolo_split_path.mkdir(exist_ok=True)
    for image_id in image_ids[split]:
        shutil.copy(object_yolo_path / f'{image_id}.jpg', object_yolo_split_path / f'{image_id}.jpg')
        shutil.copy(object_yolo_path / f'{image_id}.txt', object_yolo_split_path / f'{image_id}.txt')

# Docker 환경

```powershell
docker run -dit --name training_yolo --gpus all --ipc=host --mount type=volume,source=rico,destination=/weven/datasets --mount type=volume,source=runs,destination=/weven/runs ultralytics/ultralytics
```

- `-d` 컨테이너 백그라운드로 실행
- `-it` 터미널 연결
- `--gpus all` NVIDIA GPU 연결
- `--ipc=host` YOLO Docker 환경 추천 세팅
    
    > IPC mode에서 Shared memory 관련 세팅인데, 자세히 알 필요는 없어 보인다.
    > 
- `--mount type=volume,source=rico,destination=/weven/datasets` 데이터 관련 볼륨 마운트
- `--mount type=volume,source=runs,destination=/weven/runs` 학습 관련 볼륨 마운트
    - `runs`라는 이름의 볼륨이 없으면 자동으로 생성된다.
- `--name training_yolo` 컨테이너 이름
- `ultralytics/ultralytics` YOLOv8 이미지

# 작업 디렉토리

- `/weven/datasets` 는 `rico` 라는 이름의 Docker volume
    - RICO 데이터셋과 전처리 결과가 남아 있다.
- `/weven/runs` 는 `runs` 라는 이름의 Docker volume
    - 학습 세팅과 결과가 저장될 볼륨

```powershell
weven
├── datasets
│   ├── RICO
│   └── rico_yolo
│       ├── all
│       ├── test
│       ├── train
│       └── val
└── runs
    ├── configs
    │   └── ym_sgd_lr_5e-3_epochs_200.yaml
    └── train
        └── rico_yolo.yaml
```

## 학습 데이터셋 연결

- `/weven/runs/train` 디렉토리에 `rico_yolo.yaml` 파일을 미리 준비해 놓는다.

### rico_yolo.yaml

```yaml
train: /weven/datasets/rico_yolo/train
val: /weven/datasets/rico_yolo/val
test: /weven/datasets/rico_yolo/test
names:
  0: Advertisement
  1: Background Image
  2: Bottom Navigation
  3: Button Bar
  4: Card
  5: Checkbox
  6: Date Picker
  7: Drawer
  8: Icon
  9: Image
  10: Input
  11: List Item
  12: Map View
  13: Modal
  14: Multi-Tab
  15: Number Stepper
  16: On/Off Switch
  17: Pager Indicator
  18: Radio Button
  19: Slider
  20: Text
  21: Text Button
  22: Toolbar
  23: Video
  24: Web View
```

# YOLOv8 Training Configs

- 학습 세팅을 설정 파일 하나로 조절할 수 있어 매우 편하다.
- `/weven/runs/configs` 에 학습 세팅을 저장해둔다.

### ym_sgd_lr_5e-3_epochs_200.yaml

```yaml
task: detect
mode: train
model: yolov8m.pt
data: rico_yolo.yaml
epochs: 200
patience: 3
batch: 16
workers: 16
project: yolov8m
name: sgd_lr_5e-3_epochs_200
optimizer: SGD
lr0: 0.005
```

- 학습 결과 저장: `/weven/runs/train/{project}/{name}`

# Training

- 실행 위치: `/weven/runs/train`

In [ ]:
yolo task=detect mode=train model=yolov8m.pt data=/tf/notebook/datasets/data.yaml epochs=100 imgsz=640 batch=16

In [1]:
from ultralytics import YOLO

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

results = model.train(data='data.yaml', 
                      epoch=100,
                     imgsz=640,
                     batch=16,
                     name='yolov8m_test')



In [ ]:
!pwd

In [ ]:
!ls

# 7/27 연장 학습. 기존 학습 가중치에 추가 학습. 상황에 따라 따로... 해야할 수 도 있겠다는.... 기존데이터를 지우고 새로운 데이터들로,...

In [ ]:
TORCH_USE_CUDA_DSA

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
torch.cuda.empty_cache()

In [2]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [3]:
model = YOLO('yolov8s.pt')

model.train(data='/tf/notebook/datasets/data.yaml',
            epochs=100,  
            lr0=0.0001, 
            imgsz=320, 
            batch=16,
            name='OBElements')

New https://pypi.org/project/ultralytics/8.0.145 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.132 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/tf/notebook/datasets/data.yaml, epochs=100, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=OBElements, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnos

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.34it/s]
                   all        770       7255      0.669      0.573      0.527      0.372

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      6/100      1.34G     0.5795     0.4379     0.8677        162        320: 100%|██████████| 391/391 [00:27<00:00, 14.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.36it/s]
                   all        770       7255      0.586      0.551       0.54      0.372

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      7/100       1.3G     0.5676     0.4271     0.8633        136        320: 100%|██████████| 391/391 [00:27<00:00, 14.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.

     24/100       1.3G      0.449     0.3276     0.8347        157        320: 100%|██████████| 391/391 [00:27<00:00, 14.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.33it/s]
                   all        770       7255      0.715      0.681      0.615      0.436

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     25/100      1.28G      0.444     0.3264     0.8353        103        320: 100%|██████████| 391/391 [00:27<00:00, 14.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  4.02it/s]
                   all        770       7255      0.812      0.621      0.655      0.466

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     26/100      1.28G     0.4434     0.3234     0.8334        107        320: 100%|██████████| 391/391 [00:28<00:00, 13.52it/s]
 

     43/100      1.28G     0.3969     0.2893     0.8254        126        320: 100%|██████████| 391/391 [00:27<00:00, 14.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.30it/s]
                   all        770       7255      0.814      0.689      0.686      0.522

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     44/100      1.29G     0.3983     0.2882     0.8256        110        320: 100%|██████████| 391/391 [00:27<00:00, 14.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  4.15it/s]
                   all        770       7255      0.868      0.679      0.726      0.534

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     45/100      1.29G     0.3914      0.283     0.8221        130        320: 100%|██████████| 391/391 [00:28<00:00, 13.81it/s]
 

     62/100      1.34G     0.3608     0.2611     0.8192        132        320: 100%|██████████| 391/391 [00:27<00:00, 14.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.26it/s]
                   all        770       7255       0.89      0.713      0.768      0.566

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     63/100      1.28G     0.3615     0.2612     0.8179        156        320: 100%|██████████| 391/391 [00:28<00:00, 13.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  4.11it/s]
                   all        770       7255       0.91      0.701      0.765      0.575

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     64/100      1.29G     0.3587     0.2605     0.8184         89        320: 100%|██████████| 391/391 [00:27<00:00, 14.06it/s]
 

     81/100       1.3G     0.3369     0.2408     0.8153        147        320: 100%|██████████| 391/391 [00:28<00:00, 13.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.29it/s]
                   all        770       7255      0.956      0.712      0.786      0.599

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     82/100       1.3G     0.3322     0.2383     0.8124        138        320: 100%|██████████| 391/391 [00:28<00:00, 13.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  4.15it/s]
                   all        770       7255      0.946      0.695      0.766      0.596

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     83/100      1.29G     0.3326     0.2394     0.8131        148        320: 100%|██████████| 391/391 [00:27<00:00, 13.96it/s]
 

In [ ]:
import cv2
import numpy as np

def find_most_common_color(image_path):
    # 이미지 불러오기
    image = cv2.imread(image_path)

    # 이미지를 1차원 배열로 변환 (픽셀별 RGB 값)
    pixels = image.reshape(-1, 3)

    # RGB 값들의 빈도를 계산
    unique_colors, counts = np.unique(pixels, axis=0, return_counts=True)

    # 가장 빈도가 높은 RGB 값과 빈도 수 계산
    most_common_index = np.argmax(counts)
    most_common_color = unique_colors[most_common_index]
    most_common_count = counts[most_common_index]

    return most_common_color, most_common_count

if __name__ == "__main__":
    image_path = "/tf/notebook/test_004.png"
    most_common_color, most_common_count = find_most_common_color(image_path)
    print(f"The most common color in the image is: {most_common_color}, count: {most_common_count}")

In [ ]:
model.train(resume=True)

In [3]:
model = YOLO('/tf/notebook/runs/detect/train13/weights/last.pt')

model.train(data='/tf/notebook/datasetstest/data.yaml',
            epochs=50,
            batch=32,
            imgsz=320,
            lr0=0.001 
           )

New https://pypi.org/project/ultralytics/8.0.146 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.132 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
yolo/engine/trainer: task=detect, mode=train, model=/tf/notebook/runs/detect/train11/weights/last.pt, data=/tf/notebook/datasetstest/data.yaml, epochs=50, patience=50, batch=32, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, vi

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  2.04it/s]
                   all        818       9415      0.927      0.732      0.789      0.597

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       6/50      2.26G     0.3861     0.2609     0.8106        139        320: 100%|██████████| 209/209 [00:26<00:00,  7.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:07<00:00,  1.75it/s]
                   all        818       9415      0.957      0.751      0.802      0.632

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       7/50      2.28G     0.3828     0.2603     0.8123        105        320: 100%|██████████| 209/209 [00:27<00:00,  7.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  1.

In [3]:
model = YOLO('/tf/notebook/runs/detect/train13/weights/last.pt')
model.train(resume=True)

New https://pypi.org/project/ultralytics/8.0.146 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.132 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
yolo/engine/trainer: task=detect, mode=train, model=/tf/notebook/runs/detect/train13/weights/last.pt, data=/tf/notebook/datasetstest/data.yaml, epochs=50, patience=50, batch=32, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, vi

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  2.05it/s]
                   all        818       9415      0.947      0.789      0.892      0.691

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      44/50      2.16G     0.3844     0.2156     0.7792         77        320: 100%|██████████| 209/209 [00:26<00:00,  7.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  1.96it/s]
                   all        818       9415      0.975      0.777      0.859      0.694

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      45/50      2.04G     0.3803     0.2137     0.7795         64        320: 100%|██████████| 209/209 [00:26<00:00,  7.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  1.

In [4]:
source='/tf/notebook/test_001.png'

In [6]:
!yolo predict model=/tf/notebook/runs/detect/train13/weights/last.pt source=/tf/notebook/test_011.png 

Ultralytics YOLOv8.0.132 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
Model summary (fused): 168 layers, 11133324 parameters, 0 gradients

image 1/1 /tf/notebook/test_011.png: 128x320 2 typo_price_nums, 3 image_photos, 3 image_removeBGs, 6 typo_texts, 50.9ms
Speed: 6.4ms preprocess, 50.9ms inference, 2.3ms postprocess per image at shape (1, 3, 128, 320)
Results saved to runs/detect/predict54
